In [5]:
import json
import logging
import os
import glob
import re

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from fastprogress.fastprogress import master_bar, progress_bar
from attrdict import AttrDict

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)

from src import (
    CONFIG_CLASSES,
    TOKENIZER_CLASSES,
    MODEL_FOR_TOKEN_CLASSIFICATION,
    init_logger,
    set_seed,
    compute_metrics,
    show_ner_report
)

from processor import ner_load_and_cache_examples as load_and_cache_examples
from processor import ner_tasks_num_labels as tasks_num_labels
from processor import ner_processors as processors

logger = logging.getLogger(__name__)

from run_ner import train, evaluate

In [17]:
config_path = "../../../../data/KoELECTRA_config/naver-ner/koelectra-small-v3.json"
# Read from config file and make args
with open(config_path) as f:
    args = AttrDict(json.load(f))
logger.info("Training/evaluation parameters {}".format(args))

args.output_dir = os.path.join(args.ckpt_dir, args.output_dir)

init_logger()
set_seed(args)

02/06/2024 02:44:18 - INFO - __main__ -   Training/evaluation parameters AttrDict({'task': 'naver-ner', 'data_dir': '../../../../data/KoELECTRA', 'ckpt_dir': 'ckpt', 'train_file': 'train.tsv', 'dev_file': '', 'test_file': 'test.tsv', 'evaluate_test_during_training': True, 'eval_all_checkpoints': True, 'save_optimizer': False, 'do_lower_case': False, 'do_train': True, 'do_eval': True, 'max_seq_len': 128, 'num_train_epochs': 20, 'weight_decay': 0.0, 'gradient_accumulation_steps': 1, 'adam_epsilon': 1e-08, 'warmup_proportion': 0, 'max_steps': -1, 'max_grad_norm': 1.0, 'no_cuda': False, 'model_type': 'koelectra-small-v3', 'model_name_or_path': 'monologg/koelectra-small-v3-discriminator', 'output_dir': 'koelectra-small-v3-naver-ner-ckpt', 'seed': 42, 'train_batch_size': 32, 'eval_batch_size': 128, 'logging_steps': 1000, 'save_steps': 1000, 'learning_rate': 5e-05})


In [18]:
cache_dir = "../../../../models/huggingface"
processor = processors[args.task](args)
labels = processor.get_labels()
config = CONFIG_CLASSES[args.model_type].from_pretrained(
    args.model_name_or_path,
    num_labels=tasks_num_labels[args.task],
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=cache_dir
)
tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
    args.model_name_or_path,
    do_lower_case=args.do_lower_case,
    cache_dir=cache_dir
)
model = MODEL_FOR_TOKEN_CLASSIFICATION[args.model_type].from_pretrained(
    args.model_name_or_path,
    config=config,
    cache_dir=cache_dir
)

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# GPU or CPU
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(args.device)
print(args.device)

cuda


In [20]:
# Load dataset
train_dataset = load_and_cache_examples(args, tokenizer, mode="train") if args.train_file else None
dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev") if args.dev_file else None
test_dataset = load_and_cache_examples(args, tokenizer, mode="test") if args.test_file else None


02/06/2024 02:44:31 - INFO - processor.ner -   Creating features from dataset file at ../../../../data/KoELECTRA
02/06/2024 02:44:31 - INFO - processor.ner -   LOOKING AT ../../../../data/KoELECTRA/naver-ner/train.tsv
02/06/2024 02:44:31 - INFO - processor.ner -   금석객잔 여러분, 감사드립니다 .	ORG-B O O O
02/06/2024 02:44:31 - INFO - processor.ner -   -얼마 전에는 강남에 강천수 씨 소유의 건축물이 25억이 넘는다고 해서 말거리가 됐는데요 .	O O LOC-B PER-B O O O NUM-B O O O O O
02/06/2024 02:44:31 - INFO - processor.ner -   ” 한옥순 사장은 기념물 용적량 축소의 의미를 이렇게 표현했다 .	O PER-B CVL-B TRM-B O O O O O O
02/06/2024 02:44:31 - INFO - processor.ner -   )-경기 서반 부진했던 근거는 무엇이라 생각하는지 ?	O O O O O O O
02/06/2024 02:44:31 - INFO - processor.ner -   미다스 구귀족의 무비 아는 남자에서 불가사의한 첫 출연.	PER-B CVL-B O O O O NUM-B O
02/06/2024 02:44:31 - INFO - processor.ner -   USGA는 최하의 선수권자를 가리기 위해 엄준한 코스세팅을 한다 .	ORG-B O CVL-B O O O O O O
02/06/2024 02:44:31 - INFO - processor.ner -   남자유도 -60kg에 출정한 마터스(한국마사회)는 2회전부터 결승전까지 1 불연속 한판승을 거두는 괴력을 분휘하며 신세대를 목에 걸었다 .	CVL-B NUM-B O PER

In [21]:
if dev_dataset == None:
    args.evaluate_test_during_training = True  # If there is no dev dataset, only use testset

if args.do_train:
    global_step, tr_loss = train(args, model, train_dataset, dev_dataset, test_dataset)
    logger.info(" global_step = {}, average loss = {}".format(global_step, tr_loss))


/root/anaconda3/envs/CSU/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
02/06/2024 02:45:48 - INFO - run_ner -   ***** Running training *****
02/06/2024 02:45:48 - INFO - run_ner -     Num examples = 81000
02/06/2024 02:45:48 - INFO - run_ner -     Num Epochs = 20
02/06/2024 02:45:48 - INFO - run_ner -     Total train batch size = 32
02/06/2024 02:45:48 - INFO - run_ner -     Gradient Accumulation steps = 1
02/06/2024 02:45:48 - INFO - run_ner -     Total optimization steps = 50640
02/06/2024 02:45:48 - INFO - run_ner -     Logging steps = 1000
02/06/2024 02:45:48 - INFO - run_ner -     Save steps = 1000


02/06/2024 02:46:16 - INFO - run_ner -   ***** Running evaluation on test dataset (1000 step) *****
02/06/2024 02:46:16 - INFO - run_ner -     Num examples = 9000
02/06/2024 02:46:16 - INFO - run_ner -     Eval Batch size = 128


02/06/2024 02:46:19 - INFO - run_ner -   ***** Eval results on test dataset *****
02/06/2024 02:46:19 - INFO - run_ner -     f1 = 0.714103631697908
02/06/2024 02:46:19 - INFO - run_ner -     loss = 0.43820058459966954
02/06/2024 02:46:19 - INFO - run_ner -     precision = 0.709917215745903
02/06/2024 02:46:19 - INFO - run_ner -     recall = 0.718339715536105
/root/anaconda3/envs/CSU/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
02/06/2024 02:46:19 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.00      0.00      0.00       394
         ANM       0.78      0.31      0.44       701
         CVL       0.71      0.69      0.70      5758
         DAT       0.74      0.85      0.79      2521
         EVT    

02/06/2024 02:46:45 - INFO - run_ner -   ***** Running evaluation on test dataset (2000 step) *****
02/06/2024 02:46:45 - INFO - run_ner -     Num examples = 9000
02/06/2024 02:46:45 - INFO - run_ner -     Eval Batch size = 128


02/06/2024 02:46:48 - INFO - run_ner -   ***** Eval results on test dataset *****
02/06/2024 02:46:48 - INFO - run_ner -     f1 = 0.7626676665587238
02/06/2024 02:46:48 - INFO - run_ner -     loss = 0.3313604160933427
02/06/2024 02:46:48 - INFO - run_ner -     precision = 0.760373831775701
02/06/2024 02:46:48 - INFO - run_ner -     recall = 0.7649753829321663
02/06/2024 02:46:49 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.40      0.01      0.02       394
         ANM       0.71      0.46      0.56       701
         CVL       0.75      0.72      0.73      5758
         DAT       0.85      0.90      0.87      2521
         EVT       0.67      0.68      0.67      1094
         FLD       0.00      0.00      0.00       228
         LOC       0.73      0.74      0.73      2126
         MAT       0.00      0.00      0.00        12
         NUM       0.88      0.91      0.89      5590
         ORG       0.74      0.78      0.76      4086
 

...

02/06/2024 02:56:32 - INFO - run_ner -   ***** Running evaluation on test dataset (22000 step) *****
02/06/2024 02:56:32 - INFO - run_ner -     Num examples = 9000
02/06/2024 02:56:32 - INFO - run_ner -     Eval Batch size = 128


02/06/2024 02:56:35 - INFO - run_ner -   ***** Eval results on test dataset *****
02/06/2024 02:56:35 - INFO - run_ner -     f1 = 0.8530296592164527
02/06/2024 02:56:35 - INFO - run_ner -     loss = 0.21502089458452145
02/06/2024 02:56:35 - INFO - run_ner -     precision = 0.8481427654037246
02/06/2024 02:56:35 - INFO - run_ner -     recall = 0.8579731947483589
02/06/2024 02:56:36 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.62      0.58      0.60       394
         ANM       0.79      0.76      0.77       701
         CVL       0.82      0.84      0.83      5758
         DAT       0.91      0.94      0.92      2521
         EVT       0.75      0.78      0.76      1094
         FLD       0.63      0.63      0.63       228
         LOC       0.84      0.85      0.85      2126
         MAT       0.00      0.00      0.00        12
         NUM       0.92      0.93      0.92      5590
         ORG       0.86      0.86      0.86      4086

Epoch 9 done                                                                           


02/06/2024 02:57:02 - INFO - run_ner -   ***** Running evaluation on test dataset (23000 step) *****
02/06/2024 02:57:02 - INFO - run_ner -     Num examples = 9000
02/06/2024 02:57:02 - INFO - run_ner -     Eval Batch size = 128


02/06/2024 02:57:05 - INFO - run_ner -   ***** Eval results on test dataset *****
02/06/2024 02:57:05 - INFO - run_ner -     f1 = 0.8518493429074652
02/06/2024 02:57:05 - INFO - run_ner -     loss = 0.2183495195398868
02/06/2024 02:57:05 - INFO - run_ner -     precision = 0.8439597315436241
02/06/2024 02:57:05 - INFO - run_ner -     recall = 0.8598878555798687
02/06/2024 02:57:05 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.59      0.58      0.58       394
         ANM       0.74      0.78      0.76       701
         CVL       0.83      0.84      0.83      5758
         DAT       0.91      0.93      0.92      2521
         EVT       0.76      0.78      0.77      1094
         FLD       0.64      0.65      0.64       228
         LOC       0.84      0.85      0.84      2126
         MAT       1.00      0.08      0.15        12
         NUM       0.91      0.93      0.92      5590
         ORG       0.87      0.85      0.86      4086


In [ ]:

results = {}
if args.do_eval:
    checkpoints = list(os.path.dirname(c) for c in
                        sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True),
                                key=lambda path_with_step: list(map(int, re.findall(r"\d+", path_with_step)))[-1]))
    if not args.eval_all_checkpoints:
        checkpoints = checkpoints[-1:]
    else:
        logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce logging
        logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split("-")[-1]
        model = MODEL_FOR_TOKEN_CLASSIFICATION[args.model_type].from_pretrained(checkpoint)
        model.to(args.device)
        result = evaluate(args, model, test_dataset, mode="test", global_step=global_step)
        result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
        results.update(result)

    output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as f_w:
        if len(checkpoints) > 1:
            for key in sorted(results.keys(), key=lambda key_with_step: (
                    "".join(re.findall(r'[^_]+_', key_with_step)),
                    int(re.findall(r"_\d+", key_with_step)[-1][1:])
            )):
                f_w.write("{} = {}\n".format(key, str(results[key])))
        else:
            for key in sorted(results.keys()):
                f_w.write("{} = {}\n".format(key, str(results[key])))
